In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
def locate_goal(frame):
    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([90, 255, 255])

    # Threshold the image to get only red colors
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours are found
    if contours:
        # Find the contour with the maximum area (largest red object)
        max_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the bounding box of the contour
        x, y, w, h = cv2.boundingRect(max_contour)
        center_x = x + w // 2
        center_y = y + h // 2

        return center_x, center_y
    else:
        return None

In [3]:
def locate_thymio(frame):

    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # gaussian blur
    # hsv = cv2.GaussianBlur(hsv, (15, 15), 10)

    # # Define the lower and upper bounds for the red color in HSV
    # lower_red = np.array([0, 70, 70])
    # upper_red = np.array([10, 255, 255])

    # # Threshold the image to get only red colors
    # mask = cv2.inRange(hsv, lower_red, upper_red)

    # Define the lower and upper bounds for the red color in HSV
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])

    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    # Threshold the image to get only red colors
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Combine the two masks
    mask = cv2.bitwise_or(mask1, mask2)
    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours are found
    print(len(contours))
    if len(contours)>=2:
        # Sort contours by area
        sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

        # The largest contour is the back dot and the second largest is the front dot
        back_dot = sorted_contours[0]
        front_dot = sorted_contours[1]

        # Calculate the centers of the two dots
        back_center = np.mean(back_dot, axis=0)
        front_center = np.mean(front_dot, axis=0)

        # The orientation of the robot is the angle of the line connecting the two centers relative to the x-axis
        dx = front_center[0][0] - back_center[0][0]
        dy = front_center[0][1] - back_center[0][1]  
        orientation = np.arctan2(dy, dx)
        # print(front_center[0][0],back_center[0][0])
        # print(front_center[0][1],back_center[0][1])
        # print(dx, dy)


        return back_center, front_center, orientation, sorted_contours
    else:
        return [0,0], 0

In [4]:
def locate_table_origin(frame):
    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the white color in HSV
    lower_white = np.array([0, 0, 0])
    upper_white = np.array([180, 40, 255])  

    hsv = cv2.GaussianBlur(hsv, (15, 15), 100)

    # Threshold the image to get only blue colors
    mask = cv2.inRange(hsv, lower_white, upper_white)

    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours are found
    if contours:
        print("contours found for table:", len(contours))
        # Find the contour with the maximum area (largest white object)
        max_contour = max(contours, key=cv2.contourArea)


        cv2.drawContours(frame, [max_contour], -1, (128, 0, 128), 2)

        # Calculate the bounding box of the contour
        x, y, w, h = cv2.boundingRect(max_contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # The upper-left corner of the bounding box is the origin of the coordinate system
        origin_x = x
        origin_y = y

        return origin_x, origin_y
    else:
        return None

In [19]:
help(cv2.cvtColor)

Help on built-in function cvtColor:

cvtColor(...)
    cvtColor(src, code[, dst[, dstCn]]) -> dst
    .   @brief Converts an image from one color space to another.
    .   
    .   The function converts an input image from one color space to another. In case of a transformation
    .   to-from RGB color space, the order of the channels should be specified explicitly (RGB or BGR). Note
    .   that the default color format in OpenCV is often referred to as RGB but it is actually BGR (the
    .   bytes are reversed). So the first byte in a standard (24-bit) color image will be an 8-bit Blue
    .   component, the second byte will be Green, and the third byte will be Red. The fourth, fifth, and
    .   sixth bytes would then be the second pixel (Blue, then Green, then Red), and so on.
    .   
    .   The conventional ranges for R, G, and B channel values are:
    .   -   0 to 255 for CV_8U images
    .   -   0 to 65535 for CV_16U images
    .   -   0 to 1 for CV_32F images
    .   
    .

In [16]:
from sklearn.cluster import KMeans

def filter_contours(contours, n_clusters=2):
    # Calculate the areas of the contours
    areas = np.array([cv2.contourArea(contour) for contour in contours]).reshape(-1, 1)

    # Apply K-means clustering to the areas
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(areas)

    # Find the cluster with the largest mean area
    largest_cluster = np.argmax([np.mean(areas[kmeans.labels_ == i]) for i in range(n_clusters)])

    # Filter out contours that are not in the largest cluster
    large_contours = [contour for contour, label in zip(contours, kmeans.labels_) if label == largest_cluster]

    return large_contours

def locate_static_obstacles(frame, d):
    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the black color in HSV
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])

    # hsv = cv2.GaussianBlur(hsv, (15, 15), 0)

    # Threshold the image to get only black colors
    mask = cv2.inRange(hsv, lower_black, upper_black)

    # Threshold the image to get only blue colors
    mask = cv2.inRange(hsv, lower_black, upper_black)

    # Initialize lists to store the centers, corners, and contours of the obstacles
    obstacle_centers = []
    obstacle_corners = []
    obstacle_contours = []

    # Find contours in the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)


    # check if any contours are found
    if contours:
        # Iterate over the contours
        for contour in contours:
            # Calculate the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)
            # Enlarge the bounding rectangle by d
            x -= d
            y -= d
            w += 2*d
            h += 2*d
            print(w,h)

            # Calculate the center of the bounding rectangle
            center_x = x + w // 2
            center_y = y + h // 2

            # Add the center, corners, and contour to the respective lists
            obstacle_centers.append((center_x, center_y))
            obstacle_corners.append([(x, y), (x + w, y), (x + w, y + h), (x, y + h)])
            obstacle_contours.append(contour)

    return len(contours), obstacle_centers, obstacle_corners, obstacle_contours

# Testing locate_blue_table_origin, locate_thymio, locate_static_obstacles and locate_goal

In [18]:
# Read the image file
# frame = cv2.imread('images/test_irl6.jpeg')  # replace with your image file path

# Open the video capture
cap = cv2.VideoCapture(0)
# Read a frame from the video capture
ret, frame = cap.read()

# Apply the locate_thymio function
thymio_result = locate_thymio(frame)

# Apply the locate_static_obstacles function
obstacles_result = locate_static_obstacles(frame, 150)

# Apply the locate_goal function
goal_result = locate_goal(frame)  

# Apply the locate_blue_table_origin function
table_origin_result = locate_table_origin(frame)

if table_origin_result is not None:
    origin_x, origin_y = table_origin_result
    cv2.circle(frame, (origin_x, origin_y), 10, (255, 0, 0), -1)
else:
    origin_x, origin_y = 0, 0


# If the locate_thymio function returned a valid position
if thymio_result is not None:
    back_center, front_center, orientation, _ = thymio_result
    back_center = list(back_center[0])
    front_center = list(front_center[0])

    back_center[0] -= origin_x
    back_center[1] -= origin_y
    front_center[0] -= origin_x
    front_center[1] -= origin_y

    # Draw a circle at the position
    cv2.circle(frame, (int(back_center[0]), int(back_center[1])), 5, (0, 255, 0), -1)
    cv2.circle(frame, (int(front_center[0]), int(front_center[1])), 5, (0, 255, 0), -1)

    # Draw a line indicating the orientation
    end_x = int(back_center[0]) + 200 * np.cos(orientation)
    end_y = int(back_center[1]) + 200 * np.sin(orientation)
    cv2.line(frame, (int(back_center[0]), int(back_center[1])), (int(end_x), int(end_y)), (0, 255, 0), 2)

# If the locate_static_obstacles function found any obstacles
if obstacles_result is not None:
    num_obstacles, obstacle_centers, obstacle_corners, obstacle_contours = obstacles_result
    # Draw the obstacles
    for center, contour, corners in zip(obstacle_centers, obstacle_contours, obstacle_corners):
        center = list(center)
        # center[0] -= origin_x
        # center[1] -= origin_y
        center = tuple(center)
        cv2.circle(frame, center, 15, (0, 255, 0), -1)
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)

        # code to show corners
        corners = np.array(corners, dtype=np.int32)
        reshaped_corners = corners.reshape(-1, 1, 2)
        for corner in reshaped_corners:
            cv2.circle(frame, tuple(corner[0]), 10, (0, 255, 0), -1)
        cv2.polylines(frame, [reshaped_corners], True, (0, 255, 0), 2)


# If the locate_goal function found the goal
if goal_result is not None:
    goal_center = goal_result
    goal_center = list(goal_center)
    goal_center[0] -= origin_x
    goal_center[1] -= origin_y
    goal_center = tuple(goal_center)

    # Draw a circle at the goal position
    cv2.circle(frame, (int(goal_center[0]), int(goal_center[1])), 5, (0, 255, 0), -1)

# Save the image
cv2.imwrite('result.jpeg', frame)

# printing
print("number of obstacles:", num_obstacles)
print("static obstacle corners:", obstacle_corners)
print("thymio position and orientation:", back_center, orientation)
print("goal position:", goal_center)
print("table origin:", origin_x, origin_y)

10
787 658
625 533
738 488
contours found for table: 1
number of obstacles: 3
static obstacle corners: [[(499, 451), (1286, 451), (1286, 1109), (499, 1109)], [(1156, 99), (1781, 99), (1781, 632), (1156, 632)], [(47, -150), (785, -150), (785, 338), (47, 338)]]
thymio position and orientation: [1850.5529411764705, 185.25882352941176] -2.554166664532636
goal position: (371, 919)
table origin: 0 0


/Users/alexei.ermochkine/Desktop/ma3/mobile_robotics/venv4mobilerobotics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
def remove_shadows(frame):
    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Threshold the V channel to separate the shadows from the rest of the image
    _, shadow_mask = cv2.threshold(hsv[:,:,2], 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Apply a morphological operation to remove the shadows
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    shadow_mask = cv2.morphologyEx(shadow_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Bitwise-AND the mask with the original image
    shadow_free = cv2.bitwise_and(frame, frame, mask=shadow_mask)

    return shadow_free

In [8]:
# Open the video capture
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Apply the locate_thymio function
    result = locate_thymio(frame)

    # If the function returned a valid position
    if result is not None:
        back_center, orientation = result
        print(result)

        # Draw a circle at the position
        cv2.circle(frame, (int(back_center[0]), int(back_center[1])), 5, (0, 255, 0), -1)

        # Draw a line indicating the orientation
        # Note: You'll need to define how to calculate end_x and end_y based on the orientation
        end_x = int(back_center[0]) + 100 * np.cos(orientation)
        end_y = int(back_center[1]) + 100 * np.sin(orientation)
        cv2.line(frame, (int(back_center[0]), int(back_center[1])), (int(end_x), int(end_y)), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

200


ValueError: too many values to unpack (expected 2)

# locate red object live

In [9]:
# Example of usage:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # # Call the function to locate the center of the red object
    # red_object_center = locate_thymio(frame)

    # if red_object_center:
    #     print("Red Object Center:", red_object_center)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2023-12-07 12:27:25.005 Python[3703:139789] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 

# edges live

In [11]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    blurred_frame = cv2.GaussianBlur(gray_frame, (15, 15), 10)
    binary_image = cv2.adaptiveThreshold(blurred_frame, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    edges = cv2.Canny(binary_image, 50, 150)
    dilated = cv2.dilate(edges, (5,5), iterations=1)
    eroded = cv2.erode(dilated, (5,5), iterations=1)

    # Display the original and grayscale frames side by side
    cv2.imshow('canny', edges)

    # Find contours in the binary image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through each contour and extract coordinates
    edge_coordinates = []
    for contour in contours:
        for point in contour:
            x, y = point[0]
            edge_coordinates.append((x, y))

    # Display the original image with detected edges
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)
    cv2.imshow('Detected Edges', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Print the extracted edge coordinates
    print("Edge Coordinates:", edge_coordinates)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2023-12-07 12:26:46.267 Python[3541:127090] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Edge Coordinates: [(1456, 1067), (1456, 1073), (1459, 1066), (1459, 1067), (1458, 1068), (1459, 1068), (1457, 1052), (1457, 1053), (1456, 1054), (1456, 1055), (1455, 1056), (1456, 1056), (1456, 1054), (1457, 1053), (1456, 1049), (1455, 1050), (1455, 1052), (1455, 1050), (1456, 1049), (1457, 1050), (1457, 1049), (218, 1048), (218, 1049), (217, 1050), (217, 1052), (219, 1052), (219, 1051), (220, 1050), (220, 1048), (220, 1050), (218, 1052), (217, 1051), (217, 1050), (218, 1049), (217, 1046), (1459, 1044), (1459, 1045), (1458, 1046), (1457, 1045), (1457, 1046), (1459, 1046), (1457, 1043), (217, 1043), (1456, 1041), (1455, 1042), (1455, 1043), (1455, 1042), (1456, 1041), (1457, 1041), (218, 1041), (219, 1038), (219, 1039), (220, 1040), (220, 1042), (220, 1040), (219, 1039), (217, 1038), (217, 1039), (1459, 1034), (1459, 1035), (1458, 1036), (1457, 1035), (1457, 1036), (1459, 1036), (1458, 1031), (1458, 1032), (220, 1028), (220, 1029), (219, 1030), (219, 1032), (219, 1030), (220, 1029), (21